Scraping the visitor's tax for all the seaside resorts in the state from this site: https://www.ostseeurlaub-online.de/infos/kurtaxe/

In [1]:
from bs4 import BeautifulSoup
import requests

response = requests.get("https://www.ostseeurlaub-online.de/infos/kurtaxe/")
doc = BeautifulSoup(response.text)

In [2]:
items = doc.find_all(class_="kurtaxe-ort")
for item in items:
    print("------")
    print(item)

------
<div class="kurtaxe-ort">
<h3><a name="ahrenshoop">Ostseebad Ahrenshoop</a></h3>
<p><strong>Reisezeit A: </strong>1. Mai - 30. September<br/>
    Erwachsene: 2,50
  Euro<br/>
  Ermässigt:  2,00 Euro</p>
<p><strong>Reisezeit B: </strong>1. Oktober - 30. April<br/>
Erwachsene: 1,50
  Euro<br/>
Ermässigt:  1,00 Euro</p>
<p><strong>Sonstiges:</strong><br/>
    - Ermässigt: Schüler, Studenten, Auszubildende, Bundesfreiwilligendienstleistende, Behinderte ab 50 %<br/>
- Personen bis zur Vollendung des 18. Lebensjahr sind von der Kurtaxe befreit<br/>
- keine Kurtaxepflicht für Hunde</p>
</div>
------
<div class="kurtaxe-ort">
<h3><a name="baabe">Ostseebad Baabe</a></h3>
<p><strong>01.05.-31.10. und 23.12.-06.01.</strong><br/>
    Erwachsene: 2,30
  Euro<br/>
  Kinder (8-17 Jahre): 1,15 Euro</p>
<p><strong>07.01.-30.04. und 01.11.-22.12.</strong><br/>
Erwachsene: 1,80
  Euro<br/>
Kinder (8-17 Jahre): 0,90 Euro</p>
<p><strong>Sonstiges:</strong><br/>
    - Hunde: 0,50 Euro<br/>
    - Tage

In [3]:
for item in items:
    print("------")
    print(item.find('h3').text)
    print(item.find('p').text)

------
Ostseebad Ahrenshoop
Reisezeit A: 1. Mai - 30. September
    Erwachsene: 2,50
  Euro
  Ermässigt:  2,00 Euro
------
Ostseebad Baabe
01.05.-31.10. und 23.12.-06.01.
    Erwachsene: 2,30
  Euro
  Kinder (8-17 Jahre): 1,15 Euro
------
Ostseebad Binz
ganzjährig
  Erwachsene: 2,85 Euro
------
Ostseebad Boltenhagen
Hauptsaison vom 1. Mai bis 30. September:
  Personen ab 17 Jahren: 2,10 Euro
  Ermässigt: 1,00 Euro

------
Ostseebad Born a. Darß
ganzjährig:
    Personen ab 16 Jahre: 2,50
  Euro
  Kinder (6-15 Jahre): 1,60 Euro
------
Breege-Juliusruh
Hauptsaison vom 1. Juni bis 15. September:
  Erwachsene: 1,50 Euro
  Ermäßigt: 1,20 Euro

------
Ostseebad Dierhagen
Hauptsaison vom 1. Mai bis 31. Oktober:
  Personen ab 17 Jahre: 2,50 Euro
  Ermässigt: 2,00 Euro

------
Glowe
Hauptsaison vom 1. Juni bis 31. August:
  Erwachsene: 1,00 Euro
  Ermäßigt: 0,75 Euro

------
Graal-Müritz
Hauptsaison vom 1. Mai bis 30. September:
  Erwachsene: 2,00 Euro
  Ermäßigt: 1,00 Euro

------
Grömitz
Haupt

In [4]:
rows = []

for item in items:
    row = {}
    row['Place'] = item.find('h3').text
    row['Tax'] = item.find('p').text
    
    rows.append(row)
len(rows)

35

In [5]:
import pandas as pd

df = pd.json_normalize(rows)
df.head()

,Place,Tax
0,Ostseebad Ahrenshoop,Reisezeit A: 1. Mai - 30. September\n Erwac...
1,Ostseebad Baabe,01.05.-31.10. und 23.12.-06.01.\n Erwachsen...
2,Ostseebad Binz,"ganzjährig\n Erwachsene: 2,85 Euro"
3,Ostseebad Boltenhagen,Hauptsaison vom 1. Mai bis 30. September:\n P...
4,Ostseebad Born a. Darß,"ganzjährig:\n Personen ab 16 Jahre: 2,50\n ..."


I believe there's no way I can scrape for the prices more specifically because all the entries have the same HTML, so I'm extracting just the prices now: 

In [6]:
df['Tax'] = df['Tax'].str.extract(r'(\d+,\d{2})')

I want only the price for adults. The table displays both the tax rate for adults and for kids, but the one for adults always comes first so this should work to retrieving that one. I also compared it to the website so see whather there are any mistakes but it all looks good!

In [7]:
df.head()

,Place,Tax
0,Ostseebad Ahrenshoop,"2,50"
1,Ostseebad Baabe,"2,30"
2,Ostseebad Binz,"2,85"
3,Ostseebad Boltenhagen,"2,10"
4,Ostseebad Born a. Darß,"2,50"


It doesn't need to specify that the places are seaside resorts on the Baltic coast ('Ostseebad') every single time so I'm getting rid of that.

In [8]:
df['Place'] = df['Place'].str.replace("Ostseebad", "", regex=False)

In [9]:
df

,Place,Tax
0,Ahrenshoop,"2,50"
1,Baabe,"2,30"
2,Binz,"2,85"
3,Boltenhagen,"2,10"
4,Born a. Darß,"2,50"
5,Breege-Juliusruh,"1,50"
6,Dierhagen,"2,50"
7,Glowe,"1,00"
8,Graal-Müritz,"2,00"
9,Grömitz,"3,00"


Fixing a few typos.

In [10]:
df['Place'] = df['Place'].str.replace("Ä", "ä", regex=False)
df['Place'] = df['Place'].str.replace("bansin", "Bansin", regex=False)

In [16]:
df

,Place,Tax
0,Ahrenshoop,"2,50"
1,Baabe,"2,30"
2,Binz,"2,85"
3,Boltenhagen,"2,10"
4,Born a. Darß,"2,50"
5,Breege-Juliusruh,"1,50"
6,Dierhagen,"2,50"
7,Glowe,"1,00"
8,Graal-Müritz,"2,00"
9,Grömitz,"3,00"


Checking if the tax is a number in case I want to do calculations.

In [13]:
df.dtypes

Place    object
Tax      object
dtype: object

Before I can transcribe this into a float, I have to transform them into the English format instead of the German one.

In [21]:
df['Tax'] = df['Tax'].str.replace(",", ".", regex=False)

In [22]:
df.head()

,Place,Tax
0,Ahrenshoop,2.50
1,Baabe,2.30
2,Binz,2.85
3,Boltenhagen,2.10
4,Born a. Darß,2.50


In [23]:
df['Tax'] = df['Tax'].astype(float)

In [24]:
df.head()

,Place,Tax
0,Ahrenshoop,2.50
1,Baabe,2.30
2,Binz,2.85
3,Boltenhagen,2.10
4,Born a. Darß,2.50


Looks good now!

In [25]:
df.to_csv("Tax_clean.csv", index=False)